In [23]:
import os
from google.cloud import bigquery
from datetime import date, timedelta

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="../creds.json"
bq_client = bigquery.Client()

project = "eng-reactor-287421"

In [130]:
from google.cloud import bigquery
import pandas as pd

# Initialize the BigQuery client
client = bigquery.Client()

# Create an empty DataFrame to store the results
result_df = pd.DataFrame(columns=['par_traded_start', 'par_traded_end', 'maturity_date_start', 'maturity_date_end', 'agv', 'trade_count'])

# Define the par_traded and maturity_date buckets
par_trade_buckets = [ (0,25000), (25000, 75000), (75000, 175000), (200000, 400000), (400000, 900000), (900000, 5000000)]
maturity_buckets = [
    ("2025-09-01", "2027-09-01"),   # 2 years
    ("2028-01-01", "2030-01-01"),   # 5 years
    ("2033-01-01", "2035-01-01"),   # 10 years
    ("2038-01-01", "2040-01-01"),   # 15 years
    ("2043-01-01", "2045-01-01"),   # 20 years
    ("2050-01-01", "2055-01-01")    # 30 years
]

# Iterate through par_traded and maturity_date buckets
for par_traded_start, par_traded_end in par_trade_buckets:
    for maturity_date_start, maturity_date_end in maturity_buckets:
        # Construct and execute the query
        query = f"""
        SELECT
            AVG(a.mae) as agv,
            COUNT(a.trade_date) as trade_count
        FROM (
            SELECT
                ABS((new_ficc_ycl + new_ys_prediction) - yield) as mae,
                (new_ficc_ycl + new_ys_prediction) AS pred_yield,
                *
            FROM
                historical_predictions_test.historical_predictions_test
            ) a
        LEFT JOIN
            `auxiliary_views.materialized_trade_history` b
        ON
            a.rtrs_control_number = b.rtrs_control_number
        WHERE
            b.maturity_date > "{maturity_date_start}" AND b.maturity_date < "{maturity_date_end}"
            AND b.par_traded > {par_traded_start} AND b.par_traded < {par_traded_end}
            b.maturity_date > "2024-09-01"
            AND (b.next_call_date IS NULL
            OR b.next_call_date> "2024-09-01")
            AND (b.refund_date IS NULL
            OR b.refund_date> "2024-09-01")
          --  AND a.trade_date < "2023-09-01"
            --and a.trade_date > "2023-07-01"
            AND b.default_indicator IS FALSE
            AND b.rating_downgrade_to_junk IS FALSE
            AND (conduit_obligor_name IS NULL OR NOT REGEXP_CONTAINS(conduit_obligor_name, r'HAWAIIAN ELEC'))
            AND (conduit_obligor_name IS NULL OR NOT REGEXP_CONTAINS(conduit_obligor_name, r'TOWER HEALTH'))
            
        """
        
        # Run the query
        query_job = client.query(query)

        # Get the query results and add them to the DataFrame
        query_result = next(query_job.result())

        result_df = pd.concat([result_df, pd.DataFrame([{
            'par_traded_start': par_traded_start,
            'par_traded_end': par_traded_end,
            'maturity_date_start': maturity_date_start,
            'maturity_date_end': maturity_date_end,
            'agv': query_result.agv,
            'trade_count': query_result.trade_count
        }])], ignore_index=True)

# Print or use the result_df DataFrame as needed
print(result_df)


/var/folders/w2/bdgbts3d61132nxn6kg4v2_w0000gn/T/ipykernel_61615/3249998788.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame([{


   par_traded_start par_traded_end maturity_date_start maturity_date_end  \
0                 0          25000          2025-09-01        2027-09-01   
1                 0          25000          2028-01-01        2030-01-01   
2                 0          25000          2033-01-01        2035-01-01   
3                 0          25000          2038-01-01        2040-01-01   
4                 0          25000          2043-01-01        2045-01-01   
5                 0          25000          2050-01-01        2055-01-01   
6             25000          75000          2025-09-01        2027-09-01   
7             25000          75000          2028-01-01        2030-01-01   
8             25000          75000          2033-01-01        2035-01-01   
9             25000          75000          2038-01-01        2040-01-01   
10            25000          75000          2043-01-01        2045-01-01   
11            25000          75000          2050-01-01        2055-01-01   
12          

In [131]:
result_df

,par_traded_start,par_traded_end,maturity_date_start,maturity_date_end,agv,trade_count
0,0,25000,2025-09-01,2027-09-01,13.314745,262082
1,0,25000,2028-01-01,2030-01-01,12.583464,256081
2,0,25000,2033-01-01,2035-01-01,12.285727,166552
3,0,25000,2038-01-01,2040-01-01,12.696652,91597
4,0,25000,2043-01-01,2045-01-01,12.138726,67028
5,0,25000,2050-01-01,2055-01-01,6.792173,68995
6,25000,75000,2025-09-01,2027-09-01,11.679567,184655
7,25000,75000,2028-01-01,2030-01-01,11.881930,169634
8,25000,75000,2033-01-01,2035-01-01,11.411612,120949
9,25000,75000,2038-01-01,2040-01-01,11.250647,73003


In [141]:
par_traded_mapping = {
    0: 25,
    25000: 50,
    75000: 100,
    200000: 250,
    400000: 500,
    900000: 1000,
    5000000: 5000
}

new_df = result_df.copy()
# Update the par_traded_start values using the mapping
result_df['par_traded_start'] = result_df['par_traded_start'].map(par_traded_mapping)
result_df.rename(columns={'par_traded_start': 'par_traded', 'maturity_date_start': 'maturity'}, inplace=True)

KeyError: 'par_traded_start'

In [146]:
# Pivot the result_df DataFrame while aggregating with the mean
pivoted_df = result_df.pivot_table(index='par_traded', columns='maturity', values='agv', aggfunc='mean')

maturity_mapping = {
    '2025-09-01': 2,
    '2028-01-01': 5,
    '2033-01-01': 10,
    '2038-01-01': 15,
    '2043-01-01': 20,
    '2050-01-01': 30
}

# Rename the columns based on the mapping
pivoted_df.rename(columns=maturity_mapping, inplace=True)
# pivoted_df.columns = [f'{par_start}-{par_end}' for par_start, par_end in pivoted_df.columns]
# Print the pivoted DataFrame
pivoted_df

maturity,2,5,10,15,20,30
par_traded,,,,,,
25,13.314745,12.583464,12.285727,12.696652,12.138726,6.792173
50,11.679567,11.881930,11.411612,11.250647,9.852539,6.076007
100,10.721326,11.040935,10.815072,10.521169,9.232999,5.821986
250,10.170940,9.979162,10.219019,9.943421,8.782569,5.659410
500,10.889355,10.548600,9.466076,9.379958,8.060710,6.237497
1000,11.514567,10.489429,9.775572,8.963181,9.399448,7.673326


In [149]:
import pandas as pd
import os

# Your existing code for pivoting the DataFrame
# (I assume you've already defined 'pivoted_df')

# Define the labels and explanations
top_row_labels = ["Maturity (years)"] + list(pivoted_df.columns)
left_column_label = "Par Traded (1000s)"
explanation = "Mean average error (MAE) in yield-to-worst (YTW) in basis points (bps)"

# Create DataFrames for the left column label and explanation
left_column_label_df = pd.DataFrame([left_column_label])
explanation_df = pd.DataFrame([explanation])

# Create DataFrames for the top row labels
top_row_labels_df = pd.DataFrame([top_row_labels])

# Create a new DataFrame by concatenating the labels and pivoted data
final_df = pd.concat([left_column_label_df, explanation_df, top_row_labels_df, pivoted_df], axis=0, ignore_index=True)

# Define the file path for saving the Excel file
desktop_path = os.path.expanduser("~/Desktop")
file_name = 'ficc_ai_mae_in_bps_by_par_traded_and_maturity.xlsx'
file_path = os.path.join(desktop_path, file_name)

# Save the final DataFrame to the Excel file
final_df.to_excel(file_path, index=False, header=False)  # Skip writing headers

print(f"DataFrame saved to {file_path}")


DataFrame saved to /Users/user/Desktop/ficc_ai_mae_in_bps_by_par_traded_and_maturity.xlsx


In [138]:
import os

# Define the file path to save the Excel file on the desktop
desktop_path = os.path.expanduser("~/Desktop")
file_name = 'ficc_ai_mae_in_bps_by_par_traded_and_maturity.xlsx'
file_path = os.path.join(desktop_path, file_name)

# Save the DataFrame to the Excel file
pivoted_df.to_excel(file_path, index=False)

print(f"DataFrame saved to {file_path}")


DataFrame saved to /Users/user/Desktop/ficc_ai_mae_in_bps_by_par_traded_and_maturity.xlsx
